In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

# Veriyi yükleme
transactions = pd.read_csv('transactions.csv')  # Dosya yolunuza göre güncelleyin
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])

# Veri türü dönüşümleri
transactions['customer_id'] = transactions['customer_id'].astype(str)
transactions['product_id'] = transactions['product_id'].astype(str)

# 1. Week Number Değişkeni Oluşturma
transactions['week_number'] = (transactions['purchase_date'].dt.day - 1) // 7 + 1
transactions['month'] = transactions['purchase_date'].dt.month

# 2. Müşteri-Ürün Kombinasyonları
all_customer_product = transactions[['customer_id', 'product_id']].drop_duplicates()

# Veri türü dönüşümleri
all_customer_product['customer_id'] = all_customer_product['customer_id'].astype(str)
all_customer_product['product_id'] = all_customer_product['product_id'].astype(str)

# 3. Hedef Değişken (`week_number`)
target = transactions.groupby(['customer_id', 'product_id'])['week_number'].min().reset_index()
target.rename(columns={'week_number': 'target_week'}, inplace=True)

# Veri türü dönüşümleri
target['customer_id'] = target['customer_id'].astype(str)
target['product_id'] = target['product_id'].astype(str)

# Alınmayan ürünler için 0 atanması
all_customer_product = all_customer_product.merge(target, on=['customer_id', 'product_id'], how='left')
all_customer_product['target_week'] = all_customer_product['target_week'].fillna(0).astype(int)

# 4. Müşteri ve Ürün Bazında İstatistikler
# Müşteri istatistikleri
customer_stats = transactions.groupby('customer_id')['quantity'].agg(['mean', 'std', 'sum', 'count']).reset_index()
customer_stats.rename(columns={
    'mean': 'customer_mean',
    'std': 'customer_std',
    'sum': 'customer_total',
    'count': 'customer_frequency'
}, inplace=True)

# Ürün istatistikleri
product_stats = transactions.groupby('product_id')['quantity'].agg(['mean', 'std', 'sum', 'count']).reset_index()
product_stats.rename(columns={
    'mean': 'product_mean',
    'std': 'product_std',
    'sum': 'product_total',
    'count': 'product_frequency'
}, inplace=True)

# 5. Veri Hazırlığı
# Müşteri ve ürün istatistiklerini birleştirme
data = all_customer_product.merge(customer_stats, on='customer_id', how='left')
data = data.merge(product_stats, on='product_id', how='left')

# customer_id ve product_id'yi stringe dönüştürme (güvenlik için tekrar)
data['customer_id'] = data['customer_id'].astype(str)
data['product_id'] = data['product_id'].astype(str)

# Müşterilerin ortalama alım sıklığı 28 günden fazla ise hedef değişkeni 0 yapma
data['customer_avg_days'] = 28 / data['customer_frequency']  # Ortalama gün hesabı
data['target_week'] = np.where(data['customer_avg_days'] > 28, 0, data['target_week'])
data.drop(columns=['customer_avg_days'], inplace=True)  # Geçici sütunu kaldır

# Özellik ve hedef değişkenleri ayırma
X = data.drop(columns=['target_week'])
y = data['target_week']

# Eğitim ve test setlerinin ayrılması
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kategorik özellikler
cat_features = ['customer_id', 'product_id']

# 6. CatBoost Modeli ile Eğitim ve Tahmin
# CatBoost modelini oluşturma ve eğitme
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, verbose=100)
model.fit(X_train, y_train, cat_features=cat_features)

# Tahmin
y_pred = model.predict(X_test)

# 7. Performans Değerlendirme
# Doğruluk ve detaylı metrikler
accuracy = accuracy_score(y_test, y_pred)
print("CatBoost Model Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Test verisini yükleme
test_data = pd.read_csv('test.csv')  # Dosya yolunuza göre güncelleyin

# 1. Test Verisini Hazırlama
# customer_id ve product_id'yi stringe dönüştürme
test_data['customer_id'] = test_data['customer_id'].astype(str)
test_data['product_id'] = test_data['product_id'].astype(str)

# Ay bilgisi (örneğin, Şubat ayı için month=2 ekleniyor)
test_data['month'] = 2  # Şubat ayı olarak varsayıyoruz, projenize göre değiştirin

# İstatistikleri test verisine ekleme
test_data = test_data.merge(customer_stats, on='customer_id', how='left')
test_data = test_data.merge(product_stats, on='product_id', how='left')

# Eksik değerleri doldurma
test_data = test_data.fillna(0)

# Test verisi özelliklerini hazırlama
X_test_new = test_data.drop(columns=['id', 'prediction'], errors='ignore')

# 2. Modeli Kullanarak Tahmin Yapma
test_predictions = model.predict(X_test_new)

# Tahminleri test_data'ya ekleme
test_data['prediction'] = test_predictions

# Müşterilerin ortalama alım sıklığı 28 günden fazla ise tahminleri 0 yapma
test_data['customer_avg_days'] = 28 / test_data['customer_frequency']
test_data['prediction'] = np.where(test_data['customer_avg_days'] > 28, 0, test_data['prediction'])
test_data.drop(columns=['customer_avg_days'], inplace=True)  # Geçici sütunu kaldır

# 3. Sonuçları Kaydetme
test_data[['id',"customer_id","product_id", 'prediction']].to_csv('test_with_predictions.csv', index=False)
print("Tahminler test_with_predictions.csv dosyasına kaydedildi.")


0:	learn: 1.5862612	total: 470ms	remaining: 3m 54s
100:	learn: 1.4246542	total: 29.7s	remaining: 1m 57s
200:	learn: 1.4178590	total: 1m 10s	remaining: 1m 44s
300:	learn: 1.4136047	total: 1m 51s	remaining: 1m 13s
400:	learn: 1.4104661	total: 2m 33s	remaining: 37.9s
499:	learn: 1.4079029	total: 3m 14s	remaining: 0us
CatBoost Model Accuracy: 0.40676349519761723

Classification Report:
              precision    recall  f1-score   support

           1       0.41      0.52      0.46     44802
           2       0.40      0.47      0.43     43635
           3       0.41      0.38      0.40     40267
           4       0.41      0.33      0.37     35985
           5       0.40      0.12      0.19     14597

    accuracy                           0.41    179286
   macro avg       0.41      0.37      0.37    179286
weighted avg       0.41      0.41      0.40    179286

Tahminler test_with_predictions.csv dosyasına kaydedildi.
